In [1]:
%load_ext tensorboard


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pandas.api.types as ptypes
import re
from sklearn import preprocessing
import os
import pathlib

from tensorflow import feature_column
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
import datetime
import matplotlib
from matplotlib import pyplot

In [3]:
normalize = False
prediction_trend_data = [] 
input_numeric_columns = []
input_categorical_columns = []
output_columns = []

dfOriginal = pd.read_csv('data/revisedDataSectionsRemoved.csv', parse_dates=['date'])
df = dfOriginal
all_columns = list(df)
numeric_regex = re.compile("n_*")
input_numeric_columns = list(filter(numeric_regex.match, all_columns))
print(input_numeric_columns[0])
for item in input_numeric_columns:
    print(item)
    break
categorical_regex = re.compile("c_*")                               
input_categorical_columns = list(filter(categorical_regex.match, all_columns))
output_regex = re.compile("o_*")                               
output_columns = list(filter(output_regex.match, all_columns))
metadata_columns = ["Lat", "Lon", "regionToJoin", "date"]
df = df[input_numeric_columns+input_categorical_columns+output_columns+prediction_trend_data+metadata_columns]

if(normalize):
    #normalize data; not necessary for xgboost
    scaler = Normalizer()
    dontNormalize = input_categorical_columns + output_columns + prediction_trend_data + metadata_columns
    df.loc[:, ~df.columns.isin(dontNormalize)] = scaler.fit_transform(df.loc[:, ~df.columns.isin(dontNormalize)])

n_f_APCPsurface1HourForecast
n_f_APCPsurface1HourForecast


In [5]:
yColumnlist = []
outputList = []

In [122]:
#Work towards making classes so the code is much more concise

class Target:
    name = ''

    x_train = pd.DataFrame()
    x_test = pd.DataFrame()
    x_val = pd.DataFrame()

    y_train = pd.DataFrame()
    y_test = pd.DataFrame()
    y_val = pd.DataFrame()

    y_data = pd.DataFrame()

    y_column = []
    y_data = []
    labels = []
    def __init__(self, featureName, columnName, df):
        name = featureName
        y_column = [columnName]
        y_data = df[y_column + metadata_columns]
        x_train, x_test, y_train, y_test = train_test_split(X, y_data, stratify=y_data[columnName], test_size=0.20, random_state=1)
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify=y_train[columnName], test_size=0.20, random_state=1)

    


In [123]:
test = Target('o_day1DangerAboveTreeline', df)

In [6]:
X_n = df[input_numeric_columns]
X = X_n

yColumns = ['o_day1DangerAboveTreeline']
yColumnlist.append('o_day1DangerAboveTreeline')
yAbove=df[yColumns + metadata_columns]
X_Above_train, X_Above_test, y_Above_train, y_Above_test = train_test_split(X, yAbove, stratify=yAbove["o_day1DangerAboveTreeline"], test_size=0.20, random_state=1)
X_Above_train, X_Above_val, y_Above_train, y_Above_val = train_test_split(X_Above_train, y_Above_train, stratify=y_Above_train['o_day1DangerAboveTreeline'], test_size=0.20, random_state=1)
outputList.append(y_Above_train)

yNearColumn = ['o_day1DangerNearTreeline']
yColumnlist.append('o_day1DangerNearTreeline')
yNear=df[yNearColumn+metadata_columns]
X_Near_train, X_Near_test, y_Near_train, y_Near_test = train_test_split(X, yNear, stratify=yNear['o_day1DangerNearTreeline'], test_size=0.20, random_state=1)
X_Near_train, X_Near_val, y_Near_train, y_Near_val = train_test_split(X_Near_train, y_Near_train, stratify=y_Near_train['o_day1DangerNearTreeline'], test_size=.20, random_state=1)
outputList.append(y_Near_train)

yBelowColumn = ['o_day1DangerBelowTreeline']
yColumnlist.append('o_day1DangerBelowTreeline')
yBelow=df[yBelowColumn+metadata_columns]
X_Below_train, X_Below_test, y_Below_train, y_Below_test = train_test_split(X, yBelow, stratify=yBelow['o_day1DangerBelowTreeline'], test_size=0.20, random_state=1)
X_Below_train, X_Below_val, y_Below_train, y_Below_val = train_test_split(X_Below_train, y_Below_train, stratify=y_Below_train['o_day1DangerBelowTreeline'], test_size=0.20, random_state=1)
outputList.append(y_Below_train)

roseColumn = ['o_problemRose']
yColumnlist.append('o_problemRose')
roseInfo = df[roseColumn+metadata_columns]
x_Rose_train, x_Rose_test, y_Rose_train, y_Rose_test = train_test_split(X, roseInfo, stratify=roseInfo['o_problemRose'], test_size=.20, random_state=1)
x_Rose_train, x_Rose_val, y_Rose_train, y_Rose_val = train_test_split(x_Rose_train, y_Rose_train, stratify=y_Rose_train['o_problemRose'], test_size=.20, random_state=1)
outputList.append(y_Rose_train)

sizeColumn = ['o_problemSize']
yColumnlist.append('o_problemSize')
sizeInfo = df[sizeColumn + metadata_columns]
x_Size_train, x_Size_test, y_Size_train, y_Size_test = train_test_split(X, sizeInfo, stratify=sizeInfo['o_problemSize'], test_size=.20, random_state=1)
x_Size_train, x_Size_val, y_Size_train, y_Size_val = train_test_split(x_Size_train, y_Size_train, stratify=y_Size_train['o_problemSize'], test_size=.20, random_state=1)
outputList.append(y_Size_train)

hazardColumn = ['o_problemHazard']
yColumnlist.append('o_problemHazard')
hazardInfo = df[hazardColumn + metadata_columns]
x_Hazard_train, x_Hazard_test, y_Hazard_train, y_Hazard_test = train_test_split(X, hazardInfo, stratify=hazardInfo['o_problemHazard'], test_size=.20, random_state=1)
x_Hazard_train, x_Hazard_val, y_Hazard_train, y_Hazard_val = train_test_split(x_Hazard_train, y_Hazard_train, stratify=y_Hazard_train['o_problemHazard'], test_size=.20, random_state=1)
outputList.append(y_Hazard_train)

freqColumn = ['o_problemLikelihood']
yColumnlist.append('o_problemLikelihood')
freqInfo = df[freqColumn + metadata_columns]
x_Freq_train, x_Freq_test, y_Freq_train, y_Freq_test = train_test_split(X, freqInfo, stratify=freqInfo['o_problemLikelihood'], test_size=.20, random_state=1)
x_Freq_train, x_Freq_val, y_Freq_train, y_Freq_val = train_test_split(x_Freq_train, y_Freq_train, stratify=y_Freq_train['o_problemLikelihood'], test_size=.20, random_state=1)
outputList.append(y_Freq_train)


print(len(X_Below_train), 'train examples')
print(len(X_Below_val), 'validation examples')

y_Below_train.head()
X_Below_train.head()


12057 train examples
3015 validation examples


,n_f_APCPsurface1HourForecast,n_f_10mWindSpeed1HourForecast,n_f_APCPsurface2HourForecast,n_f_10mWindSpeed2HourForecast,n_f_APCPsurface3HourForecast,n_f_10mWindSpeed3HourForecast,n_f_APCPsurface4HourForecast,n_f_10mWindSpeed4HourForecast,n_f_APCPsurface5HourForecast,n_f_10mWindSpeed5HourForecast,...,n_r_Prev1dayMinTemp10InPast,n_r_Prev1DayPrecip10InPast,n_r_Prev24HoursPrecipAsRainTotalIn10InPast,n_r_SNOWDAS_SnowDepth_mm10InPast,n_r_SNOWDAS_SWE_mm10InPast,n_r_SNOWDAS_SnowmeltRunoff_micromm10InPast,n_r_SNOWDAS_Sublimation_micromm10InPast,n_r_SNOWDAS_SolidPrecip_kgpersquarem10InPast,n_r_SNOWDAS_LiquidPrecip_kgpersquarem10InPast,n_r_SNOWDAS_SnowpackAveTemp_k10InPast
8087,0.000,5.003431,0.000,4.714842,0.000,4.466812,0.000,3.891302,0.000,3.608659,...,17.96,0.000,0.0,179,38,0,10,0,0,265
17256,0.250,1.521547,0.375,1.485751,1.125,0.849602,2.625,0.214329,4.625,0.428978,...,28.58,90.625,0.0,0,0,0,0,0,282,0
12637,5.125,5.506617,5.375,5.804362,5.375,5.406490,5.375,5.092266,5.500,4.548136,...,28.76,59.875,0.0,37,9,133,7,3,5,273
17425,4.625,2.886101,6.625,3.994727,6.625,3.994727,8.500,4.588402,8.500,4.588402,...,21.92,339.500,0.0,543,142,1064,4,46,4,273
10113,0.000,4.083557,0.000,3.985551,0.000,3.985551,0.000,4.181581,0.000,4.181581,...,0.68,0.000,0.0,2266,665,0,0,0,0,273


In [72]:
#Function for finding labels for model prediction

def add_labels(frame, column):
    labels = []
    for index, row in frame.iterrows():
        if row[column] not in labels:
            labels.append(row[column])
    return labels
    

In [103]:
label_list = []
for i in range(0, 7):
    label_list.append(add_labels(outputList[i], yColumnlist[i]))

In [55]:
#Convert the labels to categorical labels for tensorflow

y_Below_train['o_day1DangerBelowTreeline'] = pd.Categorical(y_Below_train['o_day1DangerBelowTreeline'])
y_Below_train['o_day1DangerBelowTreeline'] = y_Below_train.o_day1DangerBelowTreeline.cat.codes

y_Below_test['o_day1DangerBelowTreeline'] = pd.Categorical(y_Below_test['o_day1DangerBelowTreeline'])
y_Below_test['o_day1DangerBelowTreeline'] = y_Below_test.o_day1DangerBelowTreeline.cat.codes

y_Below_val['o_day1DangerBelowTreeline'] = pd.Categorical(y_Below_val['o_day1DangerBelowTreeline'])
y_Below_val['o_day1DangerBelowTreeline'] = y_Below_val.o_day1DangerBelowTreeline.cat.codes

y_Above_train['o_day1DangerAboveTreeline'] = pd.Categorical(y_Above_train['o_day1DangerAboveTreeline'])
y_Above_train['o_day1DangerAboveTreeline'] = y_Above_train.o_day1DangerAboveTreeline.cat.codes

y_Above_val['o_day1DangerAboveTreeline'] = pd.Categorical(y_Above_val['o_day1DangerAboveTreeline'])
y_Above_val['o_day1DangerAboveTreeline'] = y_Above_val.o_day1DangerAboveTreeline.cat.codes

y_Near_train['o_day1DangerNearTreeline'] = pd.Categorical(y_Near_train['o_day1DangerNearTreeline'])
y_Near_train['o_day1DangerNearTreeline'] = y_Near_train.o_day1DangerNearTreeline.cat.codes

y_Near_val['o_day1DangerNearTreeline'] = pd.Categorical(y_Near_val['o_day1DangerNearTreeline'])
y_Near_val['o_day1DangerNearTreeline'] = y_Near_val.o_day1DangerNearTreeline.cat.codes

y_Rose_train['o_problemRose'] = pd.Categorical(y_Rose_train['o_problemRose'])
y_Rose_train['o_problemRose'] = y_Rose_train.o_problemRose.cat.codes

y_Rose_val['o_problemRose'] = pd.Categorical(y_Rose_val['o_problemRose'])
y_Rose_val['o_problemRose'] = y_Rose_val.o_problemRose.cat.codes

y_Hazard_train['o_problemHazard'] = pd.Categorical(y_Hazard_train['o_problemHazard'])
y_Hazard_train['o_problemHazard'] = y_Hazard_train.o_problemHazard.cat.codes

y_Hazard_val['o_problemHazard'] = pd.Categorical(y_Hazard_val['o_problemHazard'])
y_Hazard_val['o_problemHazard'] = y_Hazard_val.o_problemHazard.cat.codes

y_Size_train['o_problemSize'] = pd.Categorical(y_Size_train['o_problemSize'])
y_Size_train['o_problemSize'] = y_Size_train.o_problemSize.cat.codes

y_Size_val['o_problemSize'] = pd.Categorical(y_Size_val['o_problemSize'])
y_Size_val['o_problemSize'] = y_Size_val.o_problemSize.cat.codes

y_Freq_train['o_problemLikelihood'] = pd.Categorical(y_Freq_train['o_problemLikelihood'])
y_Freq_train['o_problemLikelihood'] = y_Freq_train.o_problemLikelihood.cat.codes

y_Freq_val['o_problemLikelihood'] = pd.Categorical(y_Freq_val['o_problemLikelihood'])
y_Freq_val['o_problemLikelihood'] = y_Freq_val.o_problemLikelihood.cat.codes




In [16]:
#data used to predict the model prediction function

test_x = X_Below_test.to_numpy()
test_unit = test_x[42]
print(test_unit)

[  0.75         5.74799913   0.75         5.43390396   1.875
   5.57379631   3.625        5.20802817   6.           4.96161835
   6.75         4.87442496   0.           4.74319438   0.
   5.03755331   0.           5.01358318   0.5          5.45263161
   1.875        5.49941623   3.75         5.76237833   4.625
   5.79010017   5.25         5.89815468   6.875        6.34529174
   8.           6.8208409   10.125        6.37065933   0.
   3.77559724   1.           5.64201582   1.25         5.86784748
   1.25         5.86272825   1.25         5.18192772   1.5
   4.89665551  38.48         6.8208409   77.          32.72
  34.7          5.5235438   61.75         0.          -3.
  -0.4         38.48         6.8208409   18.32        38.48
   6.8208409   12.02        35.6          6.08899964  18.86
   0.875        0.           2.88         0.           0.
   0.           0.         -10.           4.           0.
 273.          78.          -1.          -4.           2.
  35.6          6.08899964 

In [8]:
#splitting data into format for input to sequential NN

def df_to_dataset(dataframe, labels, labelName):
    x_input = dataframe.to_numpy()
    temp = labels.copy()
    targets = temp.pop(labelName)
    y_input = targets.values
    return x_input, y_input

In [9]:
#Create all the data sets for the NN model
#Incorperate this function into class to make it smoother

X_Below_train_ds, y_Below_train_ds = df_to_dataset(X_Below_train,  y_Below_train, 'o_day1DangerBelowTreeline')
X_Below_test_ds, y_Below_test_ds = df_to_dataset(X_Below_test, y_Below_test, 'o_day1DangerBelowTreeline')
X_Below_val_ds, y_Below_val_ds = df_to_dataset(X_Below_val, y_Below_val, 'o_day1DangerBelowTreeline')

X_Above_train_ds, y_Above_train_ds = df_to_dataset(X_Above_train, y_Above_train, 'o_day1DangerAboveTreeline')
X_Above_test_ds, y_Above_test_ds = df_to_dataset(X_Above_test, y_Above_test, 'o_day1DangerAboveTreeline')
X_Above_val_ds, y_Above_val_ds = df_to_dataset(X_Above_val, y_Above_val, 'o_day1DangerAboveTreeline')

x_Rose_train_ds, y_Rose_train_ds = df_to_dataset(x_Rose_train, y_Rose_train, 'o_problemRose')
x_Rose_val_ds, y_Rose_val_ds = df_to_dataset(x_Rose_val, y_Rose_val, 'o_problemRose')


X_Near_train_ds, y_Near_train_ds = df_to_dataset(X_Near_train, y_Near_train, 'o_day1DangerNearTreeline')
X_Near_val_ds, y_Near_val_ds = df_to_dataset(X_Near_val, y_Near_val, 'o_day1DangerNearTreeline')

x_Hazard_train_ds, y_Hazard_train_ds = df_to_dataset(x_Hazard_train, y_Hazard_train, 'o_problemHazard')
x_Hazard_val_ds, y_Hazard_val_ds = df_to_dataset(x_Hazard_val, y_Hazard_val, 'o_problemHazard')

x_Size_train_ds, y_Size_train_ds = df_to_dataset(x_Size_train, y_Size_train, 'o_problemSize')
x_Size_val_ds, y_Size_val_ds = df_to_dataset(x_Size_val, y_Size_val, 'o_problemSize')

x_Freq_train_ds, y_Freq_train_ds = df_to_dataset(x_Freq_train, y_Freq_train, 'o_problemLikelihood')
x_Freq_val_ds, y_Freq_val_ds = df_to_dataset(x_Freq_val, y_Freq_val, 'o_problemLikelihood')

In [13]:
predict_input = np.copy(X_Below_test_ds[42])

In [151]:
prediction2 = model_Below.predict(predict_Input)
print(prediction.shape)

(1, 1)


In [73]:
train_test_ds = X_Above_train_ds2.batch(1495)

In [14]:
modelBelow = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4)
])

modelBelow.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

BelowInfo = modelBelow.fit(X_Below_train_ds, y_Below_train_ds, validation_data=(X_Below_val_ds, y_Below_val_ds), epochs=50)

Train on 12057 samples, validate on 3015 samples
Epoch 1/50
12057/12057 [==============================] - 1s 75us/sample - loss: 16.0421 - accuracy: 0.6346 - val_loss: 7.7354 - val_accuracy: 0.6053
Epoch 2/50
12057/12057 [==============================] - 1s 44us/sample - loss: 4.8131 - accuracy: 0.6963 - val_loss: 3.7389 - val_accuracy: 0.7350
Epoch 3/50
12057/12057 [==============================] - 1s 54us/sample - loss: 2.4864 - accuracy: 0.7235 - val_loss: 2.3605 - val_accuracy: 0.7217
Epoch 4/50
12057/12057 [==============================] - 1s 56us/sample - loss: 1.7291 - accuracy: 0.7412 - val_loss: 2.3332 - val_accuracy: 0.7227
Epoch 5/50
12057/12057 [==============================] - 1s 50us/sample - loss: 1.3929 - accuracy: 0.7523 - val_loss: 1.6835 - val_accuracy: 0.7224
Epoch 6/50
12057/12057 [==============================] - 1s 46us/sample - loss: 1.0299 - accuracy: 0.7663 - val_loss: 1.3596 - val_accuracy: 0.7609
Epoch 7/50
12057/12057 [==============================] 

In [39]:

print(len(test_x))
test_array = np.array([test_x[42]])
print(type(test_array))
print(type(test_x))
print(len(test_x[0]))
print(len(test_array[0]))

3768
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
295
295


In [34]:
predictBelow = modelBelow.predict(test_x)

In [33]:
print(predictBelow.shape)

(3768, 4)


In [41]:
modelBelow.predict(test_array)

array([[ -1.2685325,  10.595232 ,   5.1700206, -76.73752  ]],
      dtype=float32)

In [116]:
print(BelowInfo.history)
print(BelowInfo.history['accuracy'])
BelowAccuracyList = BelowInfo.history['accuracy']
BLoss = BelowInfo.history['loss']

{'loss': [21.009884265343764, 5.433158868539884, 3.5972703239845973, 2.45016842894148, 1.6507975967421582], 'accuracy': [0.63000745, 0.70175004, 0.716845, 0.7331011, 0.7487766]}
[0.63000745, 0.70175004, 0.716845, 0.7331011, 0.7487766]


In [8]:
modelBelow.save('BelowTreeline.h5')

In [121]:
import csv
exportInfo = []


[[0.63000745, 0.70175004, 0.716845, 0.7331011, 0.7487766], [21.009884265343764, 5.433158868539884, 3.5972703239845973, 2.45016842894148, 1.6507975967421582]]


In [118]:
with open('accuracy.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(BelowExport)

In [9]:
modelNear = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4)
])

modelNear.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

NearInfo = modelNear.fit(X_Near_train_ds, y_Near_train_ds, validation_data=(X_Near_val_ds, y_Near_val_ds), epochs=50)

modelNear.save('NearTreeline.h5')

Train on 12057 samples, validate on 3015 samples
Epoch 1/50
12057/12057 [==============================] - 1s 66us/sample - loss: 23.7265 - accuracy: 0.6123 - val_loss: 6.0700 - val_accuracy: 0.7028
Epoch 2/50
12057/12057 [==============================] - 1s 44us/sample - loss: 5.6300 - accuracy: 0.6903 - val_loss: 6.6115 - val_accuracy: 0.6833
Epoch 3/50
12057/12057 [==============================] - 1s 49us/sample - loss: 3.8947 - accuracy: 0.7130 - val_loss: 3.1013 - val_accuracy: 0.7347
Epoch 4/50
12057/12057 [==============================] - 1s 54us/sample - loss: 2.4277 - accuracy: 0.7399 - val_loss: 2.1368 - val_accuracy: 0.7423
Epoch 5/50
12057/12057 [==============================] - 1s 45us/sample - loss: 1.9353 - accuracy: 0.7474 - val_loss: 2.5149 - val_accuracy: 0.6949
Epoch 6/50
12057/12057 [==============================] - 1s 48us/sample - loss: 1.5199 - accuracy: 0.7631 - val_loss: 1.6495 - val_accuracy: 0.7194
Epoch 7/50
12057/12057 [==============================] 

In [10]:
modelAbove = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4)
])

modelAbove.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

AboveInfo = modelAbove.fit(X_Above_train_ds, y_Above_train_ds, validation_data=(X_Above_val_ds, y_Above_val_ds), epochs=50)

modelAbove.save('AboveTreeline.h5')

Train on 12057 samples, validate on 3015 samples
Epoch 1/50
12057/12057 [==============================] - 1s 71us/sample - loss: 18.8458 - accuracy: 0.6048 - val_loss: 10.8470 - val_accuracy: 0.6842
Epoch 2/50
12057/12057 [==============================] - 1s 46us/sample - loss: 5.4156 - accuracy: 0.6998 - val_loss: 4.1829 - val_accuracy: 0.7068
Epoch 3/50
12057/12057 [==============================] - 1s 48us/sample - loss: 3.5347 - accuracy: 0.7285 - val_loss: 3.1345 - val_accuracy: 0.7065
Epoch 4/50
12057/12057 [==============================] - 1s 46us/sample - loss: 2.3480 - accuracy: 0.7558 - val_loss: 2.6830 - val_accuracy: 0.7506
Epoch 5/50
12057/12057 [==============================] - 1s 44us/sample - loss: 1.9432 - accuracy: 0.7625 - val_loss: 2.1017 - val_accuracy: 0.7386
Epoch 6/50
12057/12057 [==============================] - 1s 47us/sample - loss: 1.4131 - accuracy: 0.7812 - val_loss: 1.5183 - val_accuracy: 0.7711
Epoch 7/50
12057/12057 [==============================]

In [11]:
modelFreq = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])

modelFreq.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

FreqInfo = modelFreq.fit(x_Freq_train_ds, y_Freq_train_ds, validation_data=(x_Freq_val_ds, y_Freq_val_ds), epochs=50)

modelFreq.save('likelihood.h5')

Train on 12057 samples, validate on 3015 samples
Epoch 1/50
12057/12057 [==============================] - 1s 66us/sample - loss: 20.4690 - accuracy: 0.5803 - val_loss: 11.2251 - val_accuracy: 0.6342
Epoch 2/50
12057/12057 [==============================] - 1s 43us/sample - loss: 6.3378 - accuracy: 0.6667 - val_loss: 4.6343 - val_accuracy: 0.6988
Epoch 3/50
12057/12057 [==============================] - 1s 48us/sample - loss: 3.7559 - accuracy: 0.7049 - val_loss: 3.0607 - val_accuracy: 0.7274
Epoch 4/50
12057/12057 [==============================] - 1s 56us/sample - loss: 2.5648 - accuracy: 0.7254 - val_loss: 2.8334 - val_accuracy: 0.7025
Epoch 5/50
12057/12057 [==============================] - 1s 59us/sample - loss: 1.9210 - accuracy: 0.7416 - val_loss: 2.2825 - val_accuracy: 0.7174
Epoch 6/50
12057/12057 [==============================] - 1s 56us/sample - loss: 1.2865 - accuracy: 0.7648 - val_loss: 1.6120 - val_accuracy: 0.7539
Epoch 7/50
12057/12057 [==============================]

In [12]:
modelSize = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5)
])

modelSize.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

SizeInfo = modelSize.fit(x_Size_train_ds, y_Size_train_ds, validation_data=(x_Size_val_ds, y_Size_val_ds), epochs=50)

modelSize.save('size.h5')

Train on 12057 samples, validate on 3015 samples
Epoch 1/50
12057/12057 [==============================] - 1s 77us/sample - loss: 15.4865 - accuracy: 0.7421 - val_loss: 5.4457 - val_accuracy: 0.8179
Epoch 2/50
12057/12057 [==============================] - 1s 57us/sample - loss: 4.6076 - accuracy: 0.7810 - val_loss: 3.2238 - val_accuracy: 0.7900
Epoch 3/50
12057/12057 [==============================] - 1s 45us/sample - loss: 2.8103 - accuracy: 0.7900 - val_loss: 2.6880 - val_accuracy: 0.7353
Epoch 4/50
12057/12057 [==============================] - 1s 54us/sample - loss: 1.7598 - accuracy: 0.8081 - val_loss: 1.8126 - val_accuracy: 0.7433
Epoch 5/50
12057/12057 [==============================] - 1s 47us/sample - loss: 1.1857 - accuracy: 0.8232 - val_loss: 1.2828 - val_accuracy: 0.7954
Epoch 6/50
12057/12057 [==============================] - 1s 54us/sample - loss: 0.9774 - accuracy: 0.8279 - val_loss: 1.3270 - val_accuracy: 0.8315
Epoch 7/50
12057/12057 [==============================] 

In [13]:
modelHaz = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6)
])

modelHaz.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

HazInfo = modelHaz.fit(x_Hazard_train_ds, y_Hazard_train_ds,validation_data=(x_Hazard_val_ds, y_Hazard_val_ds), epochs=50)

modelHaz.save('hazard.h5')

Train on 12057 samples, validate on 3015 samples
Epoch 1/50
12057/12057 [==============================] - 1s 79us/sample - loss: 20.0154 - accuracy: 0.6983 - val_loss: 9.6243 - val_accuracy: 0.7307
Epoch 2/50
12057/12057 [==============================] - 1s 48us/sample - loss: 5.1053 - accuracy: 0.7714 - val_loss: 3.6570 - val_accuracy: 0.7284
Epoch 3/50
12057/12057 [==============================] - 1s 43us/sample - loss: 2.5609 - accuracy: 0.8151 - val_loss: 2.2601 - val_accuracy: 0.8421
Epoch 4/50
12057/12057 [==============================] - 1s 47us/sample - loss: 1.8150 - accuracy: 0.8254 - val_loss: 1.4717 - val_accuracy: 0.8186
Epoch 5/50
12057/12057 [==============================] - 1s 44us/sample - loss: 1.4617 - accuracy: 0.8394 - val_loss: 1.5243 - val_accuracy: 0.8328
Epoch 6/50
12057/12057 [==============================] - 1s 45us/sample - loss: 1.0090 - accuracy: 0.8598 - val_loss: 1.3698 - val_accuracy: 0.8093
Epoch 7/50
12057/12057 [==============================] 

In [14]:
modelRose = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32)
])

modelRose.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

RoseInfo = modelRose.fit(x_Rose_train_ds, y_Rose_train_ds, validation_data=(x_Rose_val_ds, y_Rose_val_ds), epochs=50)

modelRose.save('rose.h5')

Train on 12057 samples, validate on 3015 samples
Epoch 1/50
12057/12057 [==============================] - 1s 80us/sample - loss: 36.1969 - accuracy: 0.3335 - val_loss: 11.2063 - val_accuracy: 0.4113
Epoch 2/50
12057/12057 [==============================] - 1s 48us/sample - loss: 6.9005 - accuracy: 0.4078 - val_loss: 4.3156 - val_accuracy: 0.3602
Epoch 3/50
12057/12057 [==============================] - 1s 45us/sample - loss: 3.2064 - accuracy: 0.3964 - val_loss: 2.8126 - val_accuracy: 0.3708
Epoch 4/50
12057/12057 [==============================] - 1s 45us/sample - loss: 2.4678 - accuracy: 0.4048 - val_loss: 2.3973 - val_accuracy: 0.3934
Epoch 5/50
12057/12057 [==============================] - 1s 45us/sample - loss: 2.0275 - accuracy: 0.4312 - val_loss: 2.1093 - val_accuracy: 0.4292
Epoch 6/50
12057/12057 [==============================] - 1s 45us/sample - loss: 1.8432 - accuracy: 0.4511 - val_loss: 2.0843 - val_accuracy: 0.4252
Epoch 7/50
12057/12057 [==============================]

In [61]:
modelHaz = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8)
])

modelHaz.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [63]:
modelHaz.fit(x_Hazard_train_ds, y_Hazard_train_ds, epochs=100)

Train on 12057 samples
Epoch 1/100
12057/12057 [==============================] - 1s 73us/sample - loss: 17.1883 - accuracy: 0.6979
Epoch 2/100
12057/12057 [==============================] - 1s 46us/sample - loss: 5.4068 - accuracy: 0.7725
Epoch 3/100
12057/12057 [==============================] - 1s 42us/sample - loss: 3.1104 - accuracy: 0.8043
Epoch 4/100
12057/12057 [==============================] - 0s 40us/sample - loss: 1.8960 - accuracy: 0.8389
Epoch 5/100
12057/12057 [==============================] - 0s 39us/sample - loss: 1.4775 - accuracy: 0.8459
Epoch 6/100
12057/12057 [==============================] - 0s 40us/sample - loss: 1.0550 - accuracy: 0.8634
Epoch 7/100
12057/12057 [==============================] - 0s 40us/sample - loss: 1.0606 - accuracy: 0.8626
Epoch 8/100
12057/12057 [==============================] - 0s 40us/sample - loss: 0.7721 - accuracy: 0.8752
Epoch 9/100
12057/12057 [==============================] - 1s 43us/sample - loss: 0.7287 - accuracy: 0.8756
Epoc

In [ ]:
exportAccuracyInfo = []
exportValInfo = []
infoList = [BelowInfo, NearInfo, AboveInfo, RoseInfo, SizeInfo, FreqInfo, HazInfo]
for item in infoList:
    exportAccuracyInfo.append(item.history['accuracy'])
    exportValInfo.append(item.history['val_accuracy'])

In [ ]:
import csv

with open('accuracy.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(exportAccuracyInfo)

with open('val.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(exportValInfo)

In [ ]:
predictBelow = modelBelow.predict(test_x)
predictNear = modelNear.predict(test_x)
predictAbove = modelAbove.predict(test_x)
predictRose = modelRose.predict(test_x)
predictFreq = modelFreq.predict(test_x)
predictSize = modelSize.predict(test_x)
predictHaz = modelHaz.predict(test_x)

print(predictNear.shape)
print(predictBelow.shape)
print(predictNear.shape)
print(predictRose.shape)
print(predictFreq.shape)
print(predictSize.shape)
print(predictHaz.shape)